In [25]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from matplotlib import mlab
from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [26]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\1 December 2017 Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\1\\'

In [27]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # 如果不存在就创建

In [28]:
for fn in os.listdir(input_folder)[:10]:
    print(fn)

1P36A01R01.dat
1P36A01R02.dat
1P36A01R03.dat
1P37A01R01.dat
1P37A01R02.dat
1P37A01R03.dat
1P38A01R01.dat
1P38A01R02.dat
1P38A01R03.dat
1P39A01R01.dat


In [29]:
for fn in os.listdir(input_folder)[:10]:
    full_fn = input_folder + fn
    print(full_fn)

F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P36A01R01.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P36A01R02.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P36A01R03.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P37A01R01.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P37A01R02.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P37A01R03.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P38A01R01.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P38A01R02.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P38A01R03.dat
F:\Github\radar_HAR\Dataset_848\1 December 2017 Dataset\1P39A01R01.dat


In [30]:
def get_data_spec_MTI(input_fn):
    radar_df = pd.read_table(input_fn, sep="\n", header=None)
    center_frequency = float(radar_df[0][0])  # the carrier frequency 5.8 GHz
    sweep_time = float(radar_df[0][1])/1000  # convert to seconds (0.001 seconds)
    number_of_time_samples = float(radar_df[0][2])  # 128
    bandwidth = float(radar_df[0][3])  # 400000000Hz (400 MHz)
    sampling_frequency = number_of_time_samples / sweep_time
    '''
    record length (in seconds) = (number of measurements / 128 time samples) * sweep time of 1ms
    '''
    record_length = (len(radar_df[0][4:]) / number_of_time_samples) * sweep_time
    number_of_chirps = record_length / sweep_time  # 单分量线性调频信号的个数
    
    
    def to_complex(sample_str):
        sample_str = sample_str.replace('i', 'j')
        return complex(sample_str)
    
    
    data = radar_df[0][4:].apply(to_complex).values
    
    data_time = np.reshape(data, (int(number_of_chirps), int(number_of_time_samples)))
    data_time = np.rot90(data_time)
    
    win = np.ones((int(number_of_time_samples), data_time.shape[1]))
    win = np.ones(data_time.shape)
    fft_applied = np.fft.fftshift(np.fft.fft((data_time * win), axis=0), 0)
    data_range = fft_applied[1:int(number_of_time_samples/2), :] 
    
    
    x = data_range.shape[1]

    # set ns to nearest even number to x
    if x % 2 == 0:
        ns = x
    else:
        ns = x - 1
    
    data_range_MTI = np.zeros((data_range.shape[0], ns), dtype=np.complex128)
    (b, a) = butter(4, 0.0075, btype="high")  # 高通滤波器 high-pass Butterworth notch filter
    # Apply Filter
    for i in range(data_range.shape[0]):
        data_range_MTI[i, :ns] = lfilter(b, a, data_range[i, :ns], axis=0)
        
    # Selects range bins
    bin_indl = 10
    bin_indu = 30

    time_window_length = 200
    overlap_factor = 0.95
    overlap_length = np.round(time_window_length * overlap_factor)
    pad_factor = 4
    fft_points = pad_factor * time_window_length

    prf = 1/sweep_time
    doppler_bin = prf / fft_points
    doppler_axis = np.arange(-prf / 2, prf / 2 - doppler_bin + 1, doppler_bin)
    whole_duration = data_range_MTI.shape[1] / prf
    num_segments = np.floor((data_range_MTI.shape[1] - time_window_length) /
        (np.floor(time_window_length * (1 - overlap_factor))))
    
    data_spec_MTI=0
    for rbin in range(bin_indl-1, bin_indu):
        s, f, t = mlab.specgram(data_range_MTI[rbin, :],
                                Fs=1,
                                window=np.hamming(time_window_length),
                                noverlap=overlap_length,
                                NFFT=time_window_length, 
                                mode='complex',
                                pad_to=fft_points )

        data_spec_MTI = data_spec_MTI+abs(s)
    
    return data_spec_MTI

In [31]:
full_fn = 'F:\\Github\\radar_HAR\\Dataset_848\\1 December 2017 Dataset\\1P36A01R01.dat'
data_spec_MTI = get_data_spec_MTI(full_fn)

In [32]:
data_spec_MTI.shape

(800, 981)

In [33]:
def make_spectrograms(data_spec_MTI, window_length = 3, step_size = 100):
    window_size = 481
    iterations = data_spec_MTI.shape[1] - window_size + 1
    spectrograms = []
    for i in range(0, iterations, step_size):
        center = int(data_spec_MTI.shape[0] / 2)
        data_spec_small = data_spec_MTI[(center-150):(center+150), i:(i + window_size)]
        spectrograms.append(data_spec_small)
    return spectrograms

In [34]:
spectrograms = make_spectrograms(data_spec_MTI)

In [35]:
len(spectrograms)

6

In [36]:
spectrograms[0].shape

(300, 481)

In [37]:
fn = '1P36A01R01.dat'

In [38]:
output_fn_prefix = output_folder + fn.split('.')[0]

In [39]:
output_fn_prefix

'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01'

In [40]:
def extract_labels(fn):
    labels = fn.split('.')[0]
    activity = int(labels[0])
    person = int(labels[2:4])
    return activity, person

In [41]:
def make_image(spectrograms, output_folder, fn):
    activity, person = extract_labels(fn)  # 首先获取标签

    image_width = 75
    image_height = 125
    minimum_value = 35
    norm = colors.Normalize(vmin=minimum_value, vmax=None, clip=True)  # 划定数值最小值
    
    count = 1
    images_list = []
    activity_list = []
    person_list = []
    for spectrogram in spectrograms:
        fig = plt.figure(frameon=False)
        fig.set_size_inches(image_width, image_height)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(20 * np.log10(abs(spectrogram)), cmap='jet', norm=norm)
        image_fn = output_folder + fn.split('.')[0] + "_" + str(count)+".png"
        fig.savefig(image_fn, dpi=1)
        plt.close(fig)
        count += 1
        
        images_list.append(image_fn)
        activity_list.append(activity)
        person_list.append(person)
    
    # 返回对应的文件名称和标签
    return images_list, activity_list, person_list

In [42]:
images_list, activity_list, person_list = make_image(spectrograms, output_folder, fn)

In [43]:
images_list

['F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_1.png',
 'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_2.png',
 'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_3.png',
 'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_4.png',
 'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_5.png',
 'F:\\Github\\radar_HAR\\images_5seconds\\1\\1P36A01R01_6.png']

In [44]:
activity_list

[1, 1, 1, 1, 1, 1]

In [45]:
person_list

[36, 36, 36, 36, 36, 36]

In [46]:
def extract_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # 如果不存在就创建
    
    total_images_list = []
    total_activity_list = []
    total_person_list = []
    for fn in os.listdir(input_folder):
        print(fn)
        full_fn = input_folder + fn
        data_spec_MTI = get_data_spec_MTI(full_fn)
        spectrograms = make_spectrograms(data_spec_MTI)
        images_list, activity_list, person_list = make_image(spectrograms, output_folder, fn)
        total_images_list.extend(images_list)
        total_activity_list.extend(activity_list)
        total_person_list.extend(person_list)
    return total_images_list, total_activity_list, total_person_list

In [47]:
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)

1P36A01R01.dat
1P36A01R02.dat
1P36A01R03.dat
1P37A01R01.dat
1P37A01R02.dat
1P37A01R03.dat
1P38A01R01.dat
1P38A01R02.dat
1P38A01R03.dat
1P39A01R01.dat
1P39A01R02.dat
1P39A01R03.dat
1P40A01R01.dat
1P40A01R02.dat
1P40A01R03.dat
1P41A01R01.dat
1P41A01R02.dat
1P41A01R03.dat
1P42A01R01.dat
1P42A01R02.dat
1P42A01R03.dat
1P43A01R01.dat
1P43A01R02.dat
1P43A01R03.dat
1P44A01R01.dat
1P44A01R02.dat
1P44A01R03.dat
1P45A01R01.dat
1P45A01R02.dat
1P45A01R03.dat
1P46A01R01.dat
1P46A01R02.dat
1P46A01R03.dat
1P47A01R01.dat
1P47A01R02.dat
1P47A01R03.dat
1P50A01R01.dat
1P50A01R02.dat
1P50A01R03.dat
1P51A01R01.dat
1P51A01R02.dat
1P51A01R03.dat
1P52A01R01.dat
1P52A01R02.dat
1P52A01R03.dat
1P53A01R01.dat
1P53A01R02.dat
1P53A01R03.dat
1P53A1R01.dat
1P54A01R02.dat
1P54A01R03.dat
1P55A01R01.dat
1P55A01R02.dat
1P55A01R03.dat
1P56A01R01.dat
1P56A01R02.dat
1P56A01R03.dat
1P57A01R01.dat
1P57A01R02.dat
1P57A01R03.dat
2P36A02R01.dat
2P36A02R02.dat
2P36A02R03.dat
2P37A02R01.dat
2P37A02R02.dat
2P37A02R03.dat
2P38A02R01.

最后的图片路径和对应的标签保存为一个DataFrame并保存下来。

In [48]:
df_results1 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})

In [51]:
import pickle
pickle.dump(df_results1, open('df5_results1.pkl', 'wb'))

继续处理其他的文件夹，并把结果叠加进来：

In [52]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\2 March 2017 Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\2\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results2 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results2, open('df5_results2.pkl', 'wb'))

1P03A01R1.dat
1P03A01R2.dat
1P10A01R1.dat
1P10A01R2.dat
1P11A01R1.dat
1P11A01R2.dat
1P12A01R1.dat
1P12A01R2.dat
2P03A02R1.dat
2P03A02R2.dat
2P03A02R3.dat
2P10A02R1.dat
2P11A02R1.dat
2P11A02R2.dat
2P12A02R3.dat
2P12A02R4.dat
3P03A02R1.dat
3P03A03R2.dat
3P10A03R1.dat
3P10A03R2.dat
3P11A03R1.dat
3P11A03R2.dat
3P12A03R1.dat
3P12A03R2.dat
4P03A04R1.dat
4P03A04R2.dat
4P10A04R1.dat
4P10A04R2.dat
4P11A04R1.dat
4P11A04R2.dat
4P12A04R1.dat
4P12A04R2.dat
5P03A05R1.dat
5P03A05R2.dat
5P10A05R1.dat
5P10A05R2.dat
5P11A05R1.dat
5P11A05R2.dat
5P12A05R1.dat
5P12A05R2.dat
6P03A06R1.dat
6P03A06R2.dat
6P10A06R1.dat
6P10A06R2.dat
6P11A06R1.dat
6P11A06R2.dat
6P12A06R1.dat
6P12A06R2.dat


In [53]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\3 June 2017 Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\3\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results3 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results3, open('df5_results3.pkl', 'wb'))

1P14A01R1.dat
1P14A01R2.dat
1P14A01R3.dat
1P28A01R1.dat
1P28A01R2.dat
1P28A01R3.dat
1P29A01R1.dat
1P29A01R2.dat
1P29A01R3.dat
1P30A01R1.dat
1P30A01R2.dat
1P30A01R3.dat
1P31A01R1.dat
1P31A01R2.dat
1P31A01R3.dat
1P32A01R1.dat
1P32A01R2.dat
1P32A01R3.dat
1P33A01R1.dat
1P33A01R2.dat
1P33A01R3.dat
1P34A01R1.dat
1P34A01R2.dat
1P34A01R3.dat
1P35A01R1.dat
1P35A01R2.dat
1P35A01R3.dat
2P14A02R1.dat
2P14A02R2.dat
2P14A02R3.dat
2P28A02R1.dat
2P28A02R2.dat
2P28A02R3.dat
2P29A02R1.dat
2P29A02R2.dat
2P29A02R3.dat
2P30A02R1.dat
2P30A02R2.dat
2P30A02R3.dat
2P31A02R1.dat
2P31A02R2.dat
2P31A02R3.dat
2P32A02R1.dat
2P32A02R2.dat
2P32A02R3.dat
2P33A02R1.dat
2P33A02R2.dat
2P33A02R3.dat
2P34A02R2.dat
2P34A02R3.dat
2P34A03R1.dat
2P35A02R1.dat
2P35A02R2.dat
2P35A02R3.dat
3P14A03R1.dat
3P14A03R2.dat
3P14A03R3.dat
3P28A03R1.dat
3P28A03R2.dat
3P28A03R3.dat
3P29A03R1.dat
3P29A03R2.dat
3P29A03R3.dat
3P30A03R1.dat
3P30A03R2.dat
3P30A03R3.dat
3P31A03R1.dat
3P31A03R2.dat
3P31A03R3.dat
3P32A03R1.dat
3P32A03R2.dat
3P32A0

In [54]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\4 July 2018 Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\4\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results4 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results4, open('df5_results4.pkl', 'wb'))

1P57A01R1.dat
1P57A01R2.dat
1P57A01R3.dat
1P58A01R1.dat
1P58A01R2.dat
1P58A01R3.dat
1P59A01R1.dat
1P59A01R2.dat
1P59A01R3.dat
1P60A01R1.dat
1P60A01R2.dat
1P60A01R3.dat
1P61A01R1.dat
1P61A01R2.dat
1P61A01R3.dat
1P62A01R1.dat
1P62A01R2.dat
1P62A01R3.dat
1P63A01R1.dat
1P63A01R2.dat
1P63A01R3.dat
1P64A01R1.dat
1P64A01R2.dat
1P64A01R3.dat
1P65A01R1.dat
1P65A01R2.dat
1P65A01R3.dat
1P66A01R1.dat
1P66A01R2.dat
1P66A01R3.dat
1P67A01R1.dat
1P67A01R2.dat
1P67A01R3.dat
1P68A01R1.dat
1P68A01R2.dat
1P68A01R3.dat
1P69A01R1.dat
1P69A01R2.dat
1P69A01R3.dat
1P70A01R1.dat
1P70A01R2.dat
1P70A01R3.dat
1P71A01R1.dat
1P71A01R2.dat
1P71A01R3.dat
1P72A01R1.dat
1P72A01R2.dat
1P72A01R3.dat
2P57A02R1.dat
2P57A02R2.dat
2P57A02R3.dat
2P58A02R1.dat
2P58A02R2.dat
2P58A02R3.dat
2P59A02R1.dat
2P59A02R2.dat
2P59A02R3.dat
2P60A02R1.dat
2P60A02R2.dat
2P60A02R3.dat
2P61A02R1.dat
2P61A02R2.dat
2P61A02R3.dat
2P62A02R1.dat
2P62A02R2.dat
2P62A02R3.dat
2P63A02R1.dat
2P63A02R2.dat
2P63A02R3.dat
2P64A02R1.dat
2P64A02R2.dat
2P64A0

In [55]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\5 February 2019 UoG Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\5\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results5 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results5, open('df5_results5.pkl', 'wb'))

1P01A01R01.dat
1P01A01R02.dat
1P01A01R03.dat
1P02A01R01.dat
1P02A01R02.dat
1P02A01R03.dat
1P03A01R01.dat
1P03A01R02.dat
1P03A01R03.dat
1P04A01R01.dat
1P04A01R02.dat
1P04A01R03.dat
1P05A01R01.dat
1P05A01R02.dat
1P05A01R03.dat
1P06A01R01.dat
1P06A01R02.dat
1P06A01R03.dat
1P07A01R01.dat
1P07A01R02.dat
1P07A01R03.dat
1P08A01R01.dat
1P08A01R02.dat
1P08A01R03.dat
1P09A01R01.dat
1P09A01R02.dat
1P09A01R03.dat
1P10A01R01.dat
1P10A01R02.dat
1P10A01R03.dat
1P11A01R01.dat
1P11A01R02.dat
1P11A01R03.dat
1P12A01R01.dat
1P12A01R02.dat
1P12A01R03.dat
1P13A01R01.dat
1P13A01R02.dat
1P13A01R03.dat
1P14A01R01.dat
1P14A01R02.dat
1P14A01R03.dat
1P15A01R01.dat
1P15A01R02.dat
1P15A01R03.dat
1P16A01R01.dat
1P16A01R02.dat
1P16A01R03.dat
1P17A01R01.dat
1P17A01R02.dat
1P17A01R03.dat
2P01A02R01.dat
2P01A02R02.dat
2P01A02R03.dat
2P02A02R01.dat
2P02A02R02.dat
2P02A02R03.dat
2P03A02R01.dat
2P03A02R02.dat
2P03A02R03.dat
2P04A02R01.dat
2P04A02R02.dat
2P04A02R03.dat
2P05A02R01.dat
2P05A02R02.dat
2P05A02R03.dat
2P06A02R01

In [56]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\6 February 2019 NG Homes Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\6\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results6 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results6, open('df5_results6.pkl', 'wb'))

1P08A01R1.dat
1P08A01R2.dat
1P08A01R3.dat
1P18A01R1.dat
1P18A01R2.dat
1P18A01R3.dat
1P19A01R1.dat
1P19A01R2.dat
1P19A01R3.dat
1P20A01R1.dat
1P20A01R2.dat
1P20A01R3.dat
1P21A01R1.dat
1P21A01R2.dat
1P22A01R1.dat
1P22A01R2.dat
1P22A01R3.dat
1P23A01R1.dat
1P23A01R2.dat
1P24A01R1.dat
1P24A01R2.dat
1P24A01R3.dat
1P25A01R1.dat
1P25A01R2.dat
1P25A01R3.dat
1P26A01R1.dat
1P26A01R2.dat
1P26A01R3.dat
1P27A01R1.dat
1P27A01R2.dat
1P27A01R3.dat
1P28A01R1.dat
1P28A01R2.dat
1P28A01R3.dat
1P29A01R1.dat
1P29A01R2.dat
1P29A01R3.dat
1P30A01R1.dat
1P30A01R2.dat
1P30A01R3.dat
1P31A01R1.dat
1P31A01R2.dat
1P31A01R3.dat
1P32A01R1.dat
1P32A01R2.dat
1P32A01R3.dat
1P33A01R1.dat
1P33A01R2.dat
1P33A01R3.dat
1P34A01R1.dat
1P34A01R2.dat
1P34A01R3.dat
1P35A01R1.dat
1P35A01R2.dat
1P35A01R3.dat
1P36A01R1.dat
1P36A01R2.dat
1P36A01R3.dat
2P08A02R1.dat
2P08A02R2.dat
2P08A02R3.dat
2P18A02R1.dat
2P18A02R2.dat
2P18A02R3.dat
2P19A02R1.dat
2P19A02R2.dat
2P19A02R3.dat
2P20A02R1.dat
2P20A02R2.dat
2P20A02R3.dat
2P21A02R1.dat
2P21A0

In [57]:
input_folder = 'F:\\Github\\radar_HAR\\Dataset_848\\7 March 2019 West Cumbria Dataset\\'
output_folder = 'F:\\Github\\radar_HAR\\images_5seconds\\7\\'
total_images_list, total_activity_list, total_person_list = extract_images(input_folder, output_folder)
df_results7 = pd.DataFrame({'images': total_images_list, 'activity': total_activity_list, 'person': total_person_list})
pickle.dump(df_results7, open('df5_results7.pkl', 'wb'))

1P37A01R1.dat
1P37A01R2.dat
1P37A01R3.dat
1P38A01R1.dat
1P38A01R2.dat
1P38A01R3.dat
1P39A01R1.dat
1P39A01R2.dat
1P39A01R3.dat
1P40A01R1.dat
1P40A01R2.dat
1P40A01R3.dat
1P41A01R1.dat
1P41A01R2.dat
1P41A01R3.dat
1P42A01R1.dat
1P42A01R2.dat
1P42A01R3.dat
1P43A01R1.dat
1P43A01R2.dat
1P43A01R3.dat
1P44A01R1.dat
1P44A01R2.dat
1P44A01R3.dat
1P45A01R1.dat
1P45A01R2.dat
1P45A01R3.dat
1P46A01R1.dat
1P46A01R2.dat
1P46A01R3.dat
1P47A01R1.dat
1P47A01R2.dat
1P47A01R3.dat
1P48A01R1.dat
1P48A01R2.dat
1P48A01R3.dat
1P49A01R1.dat
1P49A01R2.dat
1P49A01R3.dat
1P50A01R1.dat
1P50A01R2.dat
1P50A01R3.dat
1P51A01R1.dat
1P51A01R2.dat
1P51A01R3.dat
1P52A01R1.dat
1P52A01R2.dat
1P52A01R3.dat
1P53A01R1.dat
1P53A01R2.dat
1P53A01R3.dat
1P54A01R1.dat
1P54A01R2.dat
1P54A01R3.dat
1P55A01R1.dat
1P55A01R2.dat
1P55A01R3.dat
1P56A01R1.dat
1P56A01R2.dat
1P56A01R3.dat
2P37A02R1.dat
2P37A02R2.dat
2P37A02R3.dat
2P38A02R1.dat
2P38A02R2.dat
2P38A02R3.dat
2P39A02R1.dat
2P39A02R2.dat
2P39A02R3.dat
2P40A02R1.dat
2P40A02R2.dat
2P40A0